In [1]:
from google.colab import drive
drive.mount('/content/drive')


KeyboardInterrupt: 

# InsightSpike-AI: Maze 51x51 Full Run (Colab Pro)

This notebook runs a production-scale 51x51 maze experiment using the L3-only paper preset and saves results to Google Drive.

Notes:
- Runtime > Change runtime type > GPU (A100 if available). CPU also works.
- Results and figures are saved under Drive.


In [ ]:
!nvidia-smi


## 1. Setup Environment


In [ ]:
import os

REPO_URL = 'https://github.com/miyauchikazuyoshi/InsightSpike-AI.git'
REPO_DIR = '/content/InsightSpike-AI'

if not os.path.exists(REPO_DIR):
    !git clone --depth 1 {REPO_URL}

%cd /content/InsightSpike-AI
!pip -q install -e .
!pip -q install networkx numpy matplotlib pandas


## 2. Configure Run Parameters


In [ ]:
import os
import time

DRIVE_ROOT = '/content/drive/MyDrive/insightspike/maze_51x51_full'
RUN_TAG = time.strftime('maze51x51_s1500_seeds13_%Y%m%d_%H%M%S')
RUN_DIR = os.path.join(DRIVE_ROOT, RUN_TAG)

MAZE_SIZE = 51
MAX_STEPS = 1500
SEEDS = 13
SEED_START = 0

SUMMARY_PATH = os.path.join(RUN_DIR, f'_{MAZE_SIZE}x{MAZE_SIZE}_s{MAX_STEPS}_l3only_summary.json')
STEPS_PATH = os.path.join(RUN_DIR, f'_{MAZE_SIZE}x{MAZE_SIZE}_s{MAX_STEPS}_l3only_steps.json')
SQLITE_PATH = os.path.join(RUN_DIR, f'mq_{MAZE_SIZE}x{MAZE_SIZE}.sqlite')
SNAPSHOT_JSON = os.path.join(RUN_DIR, f'maze_{MAZE_SIZE}x{MAZE_SIZE}.json')
FIG_DIR = os.path.join(RUN_DIR, 'figs')
LOG_DIR = os.path.join(RUN_DIR, 'logs')
MPL_DIR = os.path.join(RUN_DIR, 'mpl')

os.makedirs(RUN_DIR, exist_ok=True)
os.makedirs(FIG_DIR, exist_ok=True)
os.makedirs(LOG_DIR, exist_ok=True)
os.makedirs(MPL_DIR, exist_ok=True)

print('RUN_DIR:', RUN_DIR)
print('SUMMARY_PATH:', SUMMARY_PATH)
print('STEPS_PATH:', STEPS_PATH)


## 3. Configure Environment Variables


In [ ]:
import os

os.environ['INSIGHTSPIKE_PRESET'] = 'paper'
os.environ['INSIGHTSPIKE_LITE_MODE'] = '1'
os.environ['INSIGHTSPIKE_MIN_IMPORT'] = '1'
os.environ['INSIGHTSPIKE_LOG_DIR'] = LOG_DIR
os.environ['MPLCONFIGDIR'] = MPL_DIR


## 4. Run Experiment (L3-only)
This may take time at full scale. Results will be saved to Drive.


In [ ]:
!python experiments/maze-query-hub-prototype/tools/run_paper_l3_only.py   --maze-size {MAZE_SIZE}   --max-steps {MAX_STEPS}   --seeds {SEEDS}   --seed-start {SEED_START}   --out-root "{RUN_DIR}"   --namespace "{RUN_TAG}"   --sqlite "{SQLITE_PATH}"   --maze-snapshot-out "{SNAPSHOT_JSON}


## 5. Inspect Results


In [ ]:
!ls -lh "{RUN_DIR}" | tail -n 20


## 6. Export Metrics (Paper-Ready)


In [ ]:
!python experiments/maze-query-hub-prototype/tools/export_paper_maze.py   --summary "{SUMMARY_PATH}"   --steps "{STEPS_PATH}"   --out-json "{RUN_DIR}/maze_{MAZE_SIZE}x{MAZE_SIZE}_metrics.json"   --out-csv "{RUN_DIR}/maze_{MAZE_SIZE}x{MAZE_SIZE}_metrics.csv"   --compression-base mem


## 7. Final Visualizations


In [ ]:
import json
import os
import matplotlib.pyplot as plt

SNAPSHOT_STEP = 200

with open(SUMMARY_PATH, 'r') as f:
    summary = json.load(f)

runs = summary.get('runs', [])
if not runs:
    raise ValueError('No runs found in summary.')

run = runs[0]
seed = run.get('seed', SEED_START)

g0 = run.get('g0_series', [])
gmin = run.get('gmin_series', [])

plt.figure(figsize=(8, 3.5))
plt.plot(g0, label='g0')
plt.plot(gmin, label='gmin')
plt.xlabel('step')
plt.ylabel('g')
plt.title(f'g0/gmin time series (seed={seed})')
plt.grid(True, linewidth=0.3, alpha=0.5)
plt.legend()

g_series_path = os.path.join(FIG_DIR, f'g0_gmin_{MAZE_SIZE}x{MAZE_SIZE}_seed{seed}.png')
plt.savefig(g_series_path, dpi=180, bbox_inches='tight')
plt.show()
print('Saved:', g_series_path)

with open(STEPS_PATH, 'r') as f:
    steps = json.load(f)

step_item = next((s for s in steps if s.get('seed') == seed and s.get('step') == SNAPSHOT_STEP), None)
if step_item is None:
    candidates = [s for s in steps if s.get('seed') == seed]
    if not candidates:
        raise ValueError('No step logs found for the selected seed.')
    idx = min(SNAPSHOT_STEP, len(candidates) - 1)
    step_item = candidates[idx]

nodes = step_item.get('graph_nodes', [])
if not nodes:
    raise ValueError('No graph_nodes found in step log.')

rows = [n[0] for n in nodes]
cols = [n[1] for n in nodes]

plt.figure(figsize=(6, 6))
plt.scatter(cols, rows, s=8, alpha=0.85)
plt.gca().invert_yaxis()
plt.xlabel('col')
plt.ylabel('row')
plt.title(f'Graph growth snapshot ({MAZE_SIZE}x{MAZE_SIZE}, step={step_item.get("step")})')
plt.grid(True, linewidth=0.3, alpha=0.4)

snap_path = os.path.join(FIG_DIR, f'graph_snapshot_{MAZE_SIZE}x{MAZE_SIZE}_step{step_item.get("step")}.png')
plt.savefig(snap_path, dpi=180, bbox_inches='tight')
plt.show()
print('Saved:', snap_path)
